<a href="https://colab.research.google.com/github/mglambert/Fundamentos_procesamiento_imagenes/blob/main/1_Operaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operaciones con imagenes


### Librerias necesarias


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.morphology import remove_small_holes, erosion, square
import warnings
import cv2
warnings.filterwarnings("ignore")

### Funcion util

In [ ]:
def imshow(im, titulo='', cmap=None, rango=(None, None)):
    plt.figure()
    plt.imshow(im, vmin=rango[0], vmax=rango[1], cmap=cmap)
    plt.title(r'{}'.format(titulo), fontdict={'fontsize': 24})
    plt.axis('off')
    plt.show()

## 1º Operaciones aritmeticas

### Preambulo

Primero es necesario descargar la imagenes con las que trabajaremos. 

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jC3dYUO2KMqF66HTIi07qJkECtAeGKTd' -O im1.jpg
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1j7VIG4dJngRPRacbtS9YLiykMqTiw6bQ' -O im2.jpg

Visualizamos las imagenes y sus tamaños

In [ ]:
im1 = plt.imread('im1.jpg')
imshow(im1, 'Imagen 1')

im2 = plt.imread('im2.jpg')
imshow(im2, 'Imagen 2')

# tamanno de las imagenes
tamano1 = np.array(im1.shape)
tamano2 = np.array(im2.shape)
print('Tamano imagen 1: [{},{},{}]'.format(*tamano1))
print('Tamano imagen 2: [{},{},{}]'.format(*tamano2))

Para facilitar los ejemplos ajustamos los tamaños de las imagenes haciendo recortes.

In [ ]:
tamano2 = np.float64(tamano2)
tamano2[0:2] = np.fix(tamano1[0] * tamano2[0:2] / tamano2[0])
im2 = (255 * resize(im2, tamano2)).astype(np.uint8)
dif = tamano1[1] - tamano2[1]
im1 = im1[:, int(np.ceil(dif / 2.0)):-int(np.floor(dif / 2.0)), :].astype(np.uint8)
print('Nuevo tamano imagen 1: [{},{},{}]'.format(*im1.shape))
print('Nuevo tamano imagen 2: [{},{},{}]'.format(*im2.shape))

imshow(im1, 'Imagen 1')
imshow(im2, 'Imagen 2')

Como se observo en los tamaños de las imagenes, el tamaño esta dado por [alto, ancho, canales]. Sabemos que las imagenes jpg codifican 3 los canales R, G y B. El formato de posicion de los canales depende de la libreria utilizada para abrir las imagenes, por ejemplo matplotlib utiliza RGB mientras que OpenCV utiliza GBR. 

In [ ]:
im_aux = plt.imread('im2.jpg')
print('matplotlib')
imshow(np.hstack((im_aux[:, :, 0], im_aux[:, :, 1], im_aux[:, :, 2])), 'R $\qquad \qquad$ G $\qquad \qquad$ B', 'gray')


im_aux = cv2.imread('im2.jpg')
print('OpenCV')
imshow(np.hstack((im_aux[:, :, 0], im_aux[:, :, 1], im_aux[:, :, 2])), 'G $\qquad \qquad$ B $\qquad \qquad$ R', 'gray')

Para los ejemplos utilizaremos una mascara, de los canales observados el R es el con mejor contraste por lo que lo utilizaremos para este proposito. 

In [ ]:
mask = im2[:, :, 0] > 5
imshow(mask, 'Mascara Bruto', 'gray')
mask = remove_small_holes(mask, 50)
mask = erosion(mask, square(2))
imshow(mask, 'Mascara', 'gray')

### Multiplicacion

Las imagenes estan manejadas de forma vectorial a traves del objeto ndarray de numpy. Estos objetos utilizan las operaciones basicas, suma, multiplicacion y division de forma punto a punto. Ademas poseen ventajas operatorias cuando existe diferencias en las dimensionalidades, eso se conome como Broadcasting (pueden revisar las reglas [aqui](https://numpy.org/devdocs/user/theory.broadcasting.html)).

En el caso de la multiplicacion, esta se realiza punto a punto. Si quieramos multiplicar matrices con diferentes tamaños, como en el caso de una imagen y la mascara, es necesario agregar un eje adicional. Así los tamaños en la operacion son:

$$ [n, m, 3] * [n, m, 1] = [n, m, 3]$$

En este caso la imagen 2 se multiplica a cada canal de la imagen 1, esta operacion es conmutativa. 

In [ ]:
imshow(mask[:,:,np.newaxis]*im2, 'Im2 $\cdot$ mask')
imshow(mask[:,:,np.newaxis]*im1, 'Im1 $\cdot$ mask')

### Suma

La suma tambien se realiza punto a punto. En el caso de un array y una constante, la constante se suma a cada elemento. 

In [ ]:
im1_aux = (1-mask[:,:,np.newaxis]) * im1
im2_aux = mask[:,:,np.newaxis] * im2

im_a = im1_aux + im2_aux
imshow(im_a, 'Im2 $\cdot$ mask - (1-mask) $\cdot$ Im1')

### Division

Es equivalente a la multiplicacion, salvo que hay que tener cuidado con los elementos nulos, para lo cual agregamos un epsilon. 

In [ ]:
im_aux = np.uint8(255 * (im1/(im2_aux + np.finfo('float32').eps)))
imshow(im_aux)

### Ejemplo de aplicacion de operaciones aritmeticas - Dixon 2p

Las operaciones aritmeticas parecen triviales sin embargo, tienen aplicaciones bastante utilies. En este ejemplo veremos un calculo basico de separacion de agua y grasa en una resonancia de higado. 

En esta adquisicion se obtienen dos imagenes, una donde la señal de agua y de la grasa estan en fase y otra en la cual estan en contrafase.


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iwSxRi1rCFirt5_V4iavMqCu1FlINO1O' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iwSxRi1rCFirt5_V4iavMqCu1FlINO1O" -O Data_dixon.mat && rm -rf /tmp/cookies.txt


from scipy import io
data = io.loadmat('Data_dixon.mat')
dixon2p = data['dixon2p']
mask2 = data['mask2']

imshow(np.abs(dixon2p[:,:,0]), 'Echo 1', 'gray');
imshow(np.abs(dixon2p[:,:,1]), 'Echo 2', 'gray');

Utilizando la operacion de suma podemos separar las contribuciones de agua y grasa. 

In [ ]:
w = 0.5*(dixon2p[:,:,0]+dixon2p[:,:,1]);
imshow(np.abs(w), 'Imagen de agua', 'gray');

f = 0.5*(dixon2p[:,:,0]-dixon2p[:,:,1]);
imshow(np.abs(f), 'Imagen de grasa', 'gray');

Finalmente utilizando division podemos calcular la fraccion de grasa que compone el higado. 


In [ ]:
fg = (np.abs(f)/(np.abs(w) + np.abs(f) + np.finfo('float64').eps));
imshow(fg, 'Fraccion de grasa', 'gray');

fg = mask2*fg;
fraccion_grasa = np.sum(fg)/np.sum(mask2);
imshow(fg, 'Fraccion de grasa: {:6.5f}'.format(fraccion_grasa), 'gray');


## 2º Operaciones geometricas

En esta seccion veremos ejemplos simples de las operaciones descritas en las diapositivas del curso. 

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jBYhrV_Ntm9q8YdryY2uIGrrXFplsEOj' -O cuadro.jpg

im = plt.imread('cuadro.jpg');
imshow(im, 'Imagen Original');

### Desplazamiento

In [ ]:
T = np.float32([[1, 0, 100],
              [0, 1, 200],
              [0, 0, 1]])
[row, col, _] = im.shape
im2 = np.zeros(im.shape, dtype=np.uint8)
for i in range(3):
    im2[:,:,i] = np.uint8(cv2.warpPerspective(im[:,:,i], T, (col, row)))
imshow(im2, 'Imagen desplazada')

### Escalamiento

In [ ]:
T = np.float32([[0.5, 0, 0],
               [0, 0.5, 0],
               [0, 0, 1]])
[row, col, _] = im.shape
im2 = np.zeros(im.shape, dtype=np.uint8)
for i in range(3):
    im2[:,:,i] = np.uint8(cv2.warpPerspective(im[:,:,i], T, (col, row)))
imshow(im2, 'Imagen Escala 2')

T = np.float32([[2, 0, 0],
               [0, 2, 0],
               [0, 0, 1]])
[row, col, _] = im.shape
im2 = np.zeros(im.shape, dtype=np.uint8)
for i in range(3):
    im2[:,:,i] = np.uint8(cv2.warpPerspective(im[:,:,i], T, (col, row)))
imshow(im2, 'Imagen Escala 2')

### Rotacion

In [ ]:
angulo = 10
a = np.deg2rad(angulo)
T = np.float32([[np.cos(a), np.sin(a), 0],
               [-np.sin(a), np.cos(a), 0],
               [0, 0, 1]])
[row, col, _] = im.shape
im2 = np.zeros(im.shape, dtype=np.uint8)
for i in range(3):
    im2[:,:,i] = np.uint8(cv2.warpPerspective(im[:,:,i], T, (col, row)))
imshow(im2, 'Imagen Rotada')

### Shear vertical

In [ ]:
T = np.float32([[1, 0.1, 0],
               [0, 1, 0],
               [0, 0, 1]])
[row, col, _] = im.shape
im2 = np.zeros(im.shape, dtype=np.uint8)
for i in range(3):
    im2[:,:,i] = np.uint8(cv2.warpPerspective(im[:,:,i], T, (col, row)))
imshow(im2, 'Imagen shear vertical')

### Shear horizontal

In [ ]:
T = np.float32([[1, 0, 0],
               [0.1, 1, 0],
               [0, 0, 1]])
[row, col, _] = im.shape
im2 = np.zeros(im.shape, dtype=np.uint8)
for i in range(3):
    im2[:,:,i] = np.uint8(cv2.warpPerspective(im[:,:,i], T, (col, row)))
imshow(im2, 'Imagen shear horizontal')